In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 500

### Загрузим датасет с машинами. Цель - верно восстанавливать для каждой из них цену продажи!

In [4]:
data = pd.read_csv('autos.csv')

data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Individual,Manual,Second Owner


In [5]:
data.shape

(4340, 8)

In [6]:
### Колонка с тергетом - "selling price"

X = data.drop("selling_price", axis=1)
y = data["selling_price"]

### Будем замерять MSLE!
### Поэтому прологарифмируем таргет
### А после оптимизируем MSE

y = y.apply(np.log1p)

In [7]:
### Разделим выборку на трейн и тест!

from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

__Задание__ 

Реализуйте свой MeanTargetEncoder с добавленем некоторого шума!

Однажды в лекционном материале, обсуждая счетчики, мы говорили с вами о том, что из-за них модели могут переобучаться. Один из способов бороться с этим - валидировать расчеты среднего таргета (стратегия отложенной выборки / расчеты на кросс-валидации). Но есть еще проще!

Можно просто к значению счетчика добавить случайный шум (зашумить данные)!

Напомним, что рассчитываться новые признаки должны по такой формуле:

$$
g_j = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}{l} + C * \epsilon
$$


Пусть шум будет случайной величиной из нормального стандартного распределения, то есть $\epsilon \sim N(0, 1) $, а $ C = 0.006$.

Создавай свой класс-трансформер, наследуйтесь от классов `BaseEstimator, TransformerMixin` из `sklearn.base`. Трансформер не должен модифицировать передаваемую ему выборку inplace, а все необходимые статистики нужно считать только по обучающей выборке в методе `fit`. 
Ваш трансформер должен принимать при инициализации список из категориальных признаков и список из числовых признаков. 

Если для какого-то признака в тестовой выборке отсутствует значение, трансформер должен поставить там 0.

На выходе должен получиться датасет того же размера с измененными категориальными признаками

In [8]:
object_cols = ['name', 'year', 'fuel', 'seller_type', 'transmission', 'owner']
num_cols = ['km_driven']

X.head()

,name,year,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,141000,Diesel,Individual,Manual,Second Owner


In [33]:
from sklearn.base import BaseEstimator, TransformerMixin
import itertools

class MeanTargetEncoderNoise(BaseEstimator, TransformerMixin):
    
    
    def __init__(self, categorical, numeric):
        self.categorical = categorical
        self.numeric = numeric
    
    
    def fit(self, X, y):
        X_ = X.copy()
        y_ = y.copy()
        
        X_f = pd.concat((X_,y_), axis=1)
        
        self.mean_dict = {col:X_f.groupby(col)['selling_price'].mean() \
                          for col in self.categorical}
        
        return self
    
    def transform(self, df):
        data = df.copy()
        
        for col in self.categorical:
            data[col] = data[col].map(self.mean_dict[col]) + .006 * np.random.randn()
            data[col].fillna(0, inplace=True)
        
        return data

In [34]:
### Проверка работы трансформера

np.random.seed(1)
transformer = MeanTargetEncoderNoise(object_cols,num_cols)

transformer.fit(X_train, y_train)

train = transformer.transform(X_train)
test = transformer.transform(X_test)

test.head()

,name,year,km_driven,fuel,seller_type,transmission,owner
2761,13.677479,13.548297,15000,13.091037,12.616600,12.646961,12.960574
3210,13.050631,12.924308,70000,13.091037,12.616600,12.646961,12.447828
2606,13.133030,12.622395,90000,13.091037,12.616600,12.646961,12.960574
1030,12.395071,13.130976,41000,12.455263,13.145823,12.646961,12.960574
3942,13.388497,12.924308,71318,13.091037,13.145823,12.646961,12.960574


Обучите несколько деревьев, перебирая максимальную глубину алгоритма из списка `max_depth_list`, а остальные параметры оставьте дефолтными. Выведите лучшее значение гиперпараметра. Постройте график зависимости MSLE на тестовой выборке от значения гиперпараметра. Воспользуйтесь `Pipeline` без `GridSearch`. Проделайте то же самое с `min_samples_split`, `min_impurity_decrease`, `max_leaf_nodes`. (по 2б на каждый параметр)

In [45]:
max_depth_list = [3, 5, 8, 12]
min_samples_split_list = [10, 50, 100, 500]
min_impurity_decrease_list = [0, 0.1, 0.15, 0.2]
max_leaf_nodes_list = [100, 200, 500]

In [76]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline

np.random.seed(1)

best_score = {'mse':0,
              'max_depth':0,
              'min_samples_split':0,
              'min_impurity_decrease':0,
              'max_leaf_nodes':0}

for i in max_depth_list:
    for j in min_samples_split_list:
        for k in min_impurity_decrease_list:
            for l in max_leaf_nodes_list:
                pipe = Pipeline([
                    ('mte', MeanTargetEncoderNoise(object_cols, num_cols)),
                    ('dec_tree', DecisionTreeRegressor(max_depth=i,
                                                       min_samples_split=j,
                                                       min_impurity_decrease=k,
                                                       max_leaf_nodes=l))
                ])
                
                pipe.fit(X_train, y_train)
                
                msle = mse(y_test, pipe.predict(X_test))
                
                if best_score['mse']!=0 and best_score['mse'] > msle:
                    best_score['mse'] = msle
                    best_score['max_depth'] = i
                    best_score['min_samples_split'] = j
                    best_score['min_impurity_decrease'] = k
                    best_score['max_leaf_nodes'] = l                
                else:
                    best_score['mse'] = msle
                    best_score['max_depth'] = i
                    best_score['min_samples_split'] = j
                    best_score['min_impurity_decrease'] = k
                    best_score['max_leaf_nodes'] = l


print(best_score)

{'mse': 0.5204871413036704, 'max_depth': 12, 'min_samples_split': 500, 'min_impurity_decrease': 0.2, 'max_leaf_nodes': 500}


Подберите лучшую комбинацию параметров, используя `GridSearchCV` и набор массивов значений параметров из предыдущего задания. Для лучшей комбинации посчитайте MSLE на тестовой выборке. Получились ли лучшие параметры такими же, как если бы вы подбирали их по-отдельности при остальных гиперпараметрах по умолчанию (предыдущее задание)? (2б)

In [77]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "dec_tree__max_depth": [3, 5, 8, 12],
    "dec_tree__min_samples_split": [10, 50, 100, 500],
    "dec_tree__min_impurity_decrease": [0, 0.1, 0.15, 0.2],
    "dec_tree__max_leaf_nodes": [100, 200, 500]
}
np.random.seed(1)

search = GridSearchCV(pipe, param_grid=param_grid, scoring='neg_mean_squared_error')

search.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('mte',
                                        MeanTargetEncoderNoise(categorical=['name',
                                                                            'year',
                                                                            'fuel',
                                                                            'seller_type',
                                                                            'transmission',
                                                                            'owner'],
                                                               numeric=['km_driven'])),
                                       ('dec_tree',
                                        DecisionTreeRegressor(max_depth=12,
                                                              max_leaf_nodes=500,
                                                              min_impurity_decrease=0.2,
                                                              min_samples_split=500))]),
             param_grid={'dec_tree__max_depth': [3, 5, 8, 12],
                         'dec_tree__max_leaf_nodes': [100, 200, 500],
                         'dec_tree__min_impurity_decrease': [0, 0.1, 0.15, 0.2],
                         'dec_tree__min_samples_split': [10, 50, 100, 500]},
             scoring='neg_mean_squared_error')

In [78]:
search.best_params_

{'dec_tree__max_depth': 5,
 'dec_tree__max_leaf_nodes': 100,
 'dec_tree__min_impurity_decrease': 0.2,
 'dec_tree__min_samples_split': 50}

In [80]:
mse(y_test, search.predict(X_test))

0.520487141303621